In [1]:
import torch
import pandas as pd
from torch_geometric.data import Data

#NN
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.optim import Adam

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))  # Adjust as needed
from config import DATAPATH, SAMPLE_DATAPATH

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix

In [3]:
from tqdm import tqdm

In [4]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [37]:
# Load the entire dataset
df = pd.read_csv(DATAPATH)

In [ ]:
# Filter by data range
# Node feature saved for 2022-12-31
df = df[df['Date'] < '2022-12-31']

In [39]:
df.head()

,Time,Date,Sender_account,Receiver_account,Amount,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,Laundering_type
0,10:35:19,2022-10-07,8724731955,2769355426,1459.15,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits
1,10:35:20,2022-10-07,1491989064,8401255335,6019.64,UK pounds,Dirham,UK,UAE,Cross-border,0,Normal_Fan_Out
2,10:35:20,2022-10-07,287305149,4404767002,14328.44,UK pounds,UK pounds,UK,UK,Cheque,0,Normal_Small_Fan_Out
3,10:35:21,2022-10-07,5376652437,9600420220,11895.00,UK pounds,UK pounds,UK,UK,ACH,0,Normal_Fan_In
4,10:35:21,2022-10-07,9614186178,3803336972,115.25,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits


In [40]:
# Add and delete columns
df['DateTime'] = pd.to_datetime(df["Date"] + ' ' + df["Time"], format='%Y-%m-%d %H:%M:%S')
df['Hour'] = pd.to_datetime(df['DateTime']).dt.hour

df['Date_Year'] = pd.to_datetime(df['DateTime']).dt.year
df['Date_Month'] = pd.to_datetime(df['DateTime']).dt.month
df['Date_Day'] = pd.to_datetime(df['DateTime']).dt.day

# df.drop(columns=['Laundering_type'], inplace=True)
df.drop(columns=['Time', 'Date'], inplace=True)

In [41]:
# Check class distribution
print(f"Suspicious: {df['Is_laundering'].sum()} ({100*df['Is_laundering'].mean():.2f}%)")
print(f"Normal: {(~df['Is_laundering'].astype(bool)).sum()}")

Suspicious: 2386 (0.10%)
Normal: 2478422


In [42]:
label_encoders = {}
categorical_cols = ['Payment_type', 'Sender_bank_location', 'Receiver_bank_location']

for col in categorical_cols:
    le = LabelEncoder()
    df[f'{col}_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# high risk countries
high_risk_countries = ['Mexico', 'Turkey', 'Morocco', 'UAE']  # Replace with actual countries
df['high_risk_country'] = (
    df['Sender_bank_location'].isin(high_risk_countries) | 
    df['Receiver_bank_location'].isin(high_risk_countries)
).astype(int)

# cross border transaction
df['cross_border'] = df['Payment_type'].apply(lambda x: 1 if x == 'Cross-border' else 0)    

# Create account mapping (account name -> number)
all_accounts = pd.concat([df['Sender_account'], df['Receiver_account']]).unique()
account_to_idx = {acc: idx for idx, acc in enumerate(all_accounts)}
print(f"Total accounts (nodes): {len(all_accounts)}")

Total accounts (nodes): 477061


In [43]:
# Build edge index (who sends to whom)
edge_index = torch.tensor([
    [account_to_idx[sender] for sender in df['Sender_account']],
    [account_to_idx[receiver] for receiver in df['Receiver_account']]
], dtype=torch.long)

print(f"Total transactions (edges): {edge_index.shape[1]}")

Total transactions (edges): 2480808


In [12]:
# print("\n=== STEP 3: Creating Node Features ===")

# def compute_node_features(df, account_list, account_to_idx):
#     """Compute features for each account"""
#     features = []

#     for account in tqdm(account_list):
#         # Get transactions
#         sent = df[df['Sender_account'] == account]
#         received = df[df['Receiver_account'] == account]
        
#         # Compute simple statistics
#         features.append([
#             len(sent),                                    # number of outgoing transactions
#             len(received),                                # number of incoming transactions
#             sent['Amount'].sum() if len(sent) > 0 else 0, # total sent
#             sent['Amount'].max() if len(sent) > 0 else 0, # max sent
#             sent['Amount'].min() if len(sent) > 0 else 0, # min sent
#             sent['Amount'].std() if len(sent) > 1 else 0, # std dev sent
#             received['Amount'].sum() if len(received) > 0 else 0, # total received
#             received['Amount'].max() if len(received) > 0 else 0, # max received
#             received['Amount'].min() if len(received) > 0 else 0, # min received
#             received['Amount'].std() if len(received) > 1 else 0, # std dev received
#             sent['Amount'].mean() if len(sent) > 0 else 0, # average sent
#             sent['Receiver_account'].nunique() if len(sent) > 0 else 0, # number of receivers
#             received['Sender_account'].nunique() if len(received) > 0 else 0, # number of senders
#         ])
    
#     return np.array(features, dtype=np.float32)

# node_features = compute_node_features(df, all_accounts, account_to_idx)
# print(f"Node feature shape: {node_features.shape}")


# np.save('node_features.npy', node_features)
node_features = np.load('node_features.npy')
node_features.shape

(477061, 13)

In [13]:
# Normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
node_features = scaler.fit_transform(node_features)

In [14]:
x = torch.tensor(node_features, dtype=torch.float)

In [ ]:
print("\n=== STEP 4: Creating Edge Features ===")

edge_features = df[['Amount', 'Payment_type_encoded', 
                    'Sender_bank_location_encoded', 
                    'Receiver_bank_location_encoded', 'high_risk_country']].values

# Add cyclical time features
hour = df['Hour'].values
edge_features = np.hstack((edge_features, np.sin(2 * np.pi * hour.reshape(-1, 1)/24)))
edge_features = np.hstack((edge_features, np.cos(2 * np.pi * hour.reshape(-1, 1)/24)))

month = df['Date_Month'].values
edge_features = np.hstack((edge_features, np.sin(2 * np.pi * month.reshape(-1, 1)/12)))
edge_features = np.hstack((edge_features, np.cos(2 * np.pi * month.reshape(-1, 1)/12)))

day = df['Date_Day'].values
edge_features = np.hstack((edge_features, np.sin(2 * np.pi * day.reshape(-1, 1)/31)))
edge_features = np.hstack((edge_features, np.cos(2 * np.pi * day.reshape(-1, 1)/31)))

minute = df['DateTime'].dt.minute.values
edge_features = np.hstack((edge_features, np.sin(2 * np.pi * minute.reshape(-1, 1)/60)))
edge_features = np.hstack((edge_features, np.cos(2 * np.pi * minute.reshape(-1, 1)/60)))

edge_attr = torch.tensor(edge_features, dtype=torch.float)

print(f"Edge feature shape: {edge_attr.shape}")



=== STEP 4: Creating Edge Features ===
Edge feature shape: torch.Size([2480808, 13])


In [16]:
print("\n=== STEP 5: Creating Labels ===")

y = torch.tensor(df['Is_laundering'].values, dtype=torch.long)
print(f"Labels shape: {y.shape}")


=== STEP 5: Creating Labels ===
Labels shape: torch.Size([2480808])


In [17]:
# Delete dataframe to save memory
del df

In [18]:
print("\n=== STEP 6: Creating Graph ===")

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
data = data.to(device)

print(f"Graph created:")
print(f"  Nodes: {data.num_nodes}")
print(f"  Edges: {data.num_edges}")
print(f"  Node features: {data.x.shape[1]}")
print(f"  Edge features: {data.edge_attr.shape[1]}")


=== STEP 6: Creating Graph ===
Graph created:
  Nodes: 477061
  Edges: 2480808
  Node features: 13
  Edge features: 13


In [19]:
print("\n=== STEP 7: Splitting Data Temporally ===")

num_edges = data.num_edges

# TEMPORAL SPLIT: Train on past, validate on recent, test on future
# Split indices: 70% train, 15% val, 15% test
train_size = int(0.80 * num_edges)
val_size = int(0.10 * num_edges)

train_idx = np.arange(0, train_size)
val_idx = np.arange(train_size, train_size + val_size)
test_idx = np.arange(train_size + val_size, num_edges)

print(f"\nTemporal Split:")
print(f"  Train: indices 0 to {train_size-1}")
print(f"  Val:   indices {train_size} to {train_size + val_size - 1}")
print(f"  Test:  indices {train_size + val_size} to {num_edges-1}")

# Create masks
train_mask = torch.zeros(num_edges, dtype=torch.bool, device=device)
val_mask = torch.zeros(num_edges, dtype=torch.bool, device=device)
test_mask = torch.zeros(num_edges, dtype=torch.bool, device=device)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

print(f"\nSplit sizes:")
print(f"  Train: {train_mask.sum()} ({100*train_mask.sum()/num_edges:.1f}%)")
print(f"  Val: {val_mask.sum()} ({100*val_mask.sum()/num_edges:.1f}%)")
print(f"  Test: {test_mask.sum()} ({100*test_mask.sum()/num_edges:.1f}%)")

# Check class distribution in each split
print(f"\nSuspicious transactions per split:")
print(f"  Train: {data.y[train_mask].sum()}/{train_mask.sum()} ({100*data.y[train_mask].float().mean():.3f}%)")
print(f"  Val: {data.y[val_mask].sum()}/{val_mask.sum()} ({100*data.y[val_mask].float().mean():.3f}%)")
print(f"  Test: {data.y[test_mask].sum()}/{test_mask.sum()} ({100*data.y[test_mask].float().mean():.3f}%)")

# Warning if test set has very different distribution
train_suspicious_rate = data.y[train_mask].float().mean()
test_suspicious_rate = data.y[test_mask].float().mean()

if abs(train_suspicious_rate - test_suspicious_rate) > 0.001:  # More than 0.1% difference
    print("\n⚠ WARNING: Test set has different suspicious rate than training!")
    print("  This is realistic (fraud patterns change over time)")
    print("  But it may affect performance metrics")



=== STEP 7: Splitting Data Temporally ===

Temporal Split:
  Train: indices 0 to 1984645
  Val:   indices 1984646 to 2232725
  Test:  indices 2232726 to 2480807

Split sizes:
  Train: 1984646 (80.0%)
  Val: 248080 (10.0%)
  Test: 248082 (10.0%)

Suspicious transactions per split:
  Train: 1926/1984646 (0.097%)
  Val: 306/248080 (0.123%)
  Test: 154/248082 (0.062%)


In [20]:
print("\n=== STEP 8: Defining Model ===")

class SimpleGAT(nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_dim=64, num_heads=4):
        super().__init__()
        
        # Encode node and edge features
        self.node_encoder = nn.Linear(num_node_features, hidden_dim)
        self.edge_encoder = nn.Linear(num_edge_features, hidden_dim)
        
        # GAT layers
        self.gat1 = GATConv(hidden_dim, hidden_dim, heads=num_heads, concat=True, edge_dim=hidden_dim)
        self.gat2 = GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads, concat=False, edge_dim=hidden_dim)
        
        # Classifier for edges
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2 + hidden_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)  # 2 classes: normal, suspicious
        )
    
    def forward(self, x, edge_index, edge_attr):
        # Encode
        x = F.relu(self.node_encoder(x))
        edge_embed = F.relu(self.edge_encoder(edge_attr))
        
        # GAT layer 1
        x = self.gat1(x, edge_index, edge_attr=edge_embed)
        x = F.relu(x)
        x = F.dropout(x, p=0.3, training=self.training)

        # GAT layer 2
        x = self.gat2(x, edge_index, edge_attr=edge_embed)
        x = F.relu(x)
        
        # For each edge, get sender and receiver embeddings
        sender_emb = x[edge_index[0]]
        receiver_emb = x[edge_index[1]]
        
        # Combine with edge features
        edge_features = torch.cat([sender_emb, receiver_emb, edge_embed], dim=1)
        
        # Classify
        out = self.classifier(edge_features)
        return out
    
# Create model
model = SimpleGAT(
    num_node_features=data.x.shape[1],
    num_edge_features=data.edge_attr.shape[1],
    hidden_dim=64,
    num_heads=4
).to(device)

print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")


=== STEP 8: Defining Model ===
SimpleGAT(
  (node_encoder): Linear(in_features=13, out_features=64, bias=True)
  (edge_encoder): Linear(in_features=13, out_features=64, bias=True)
  (gat1): GATConv(64, 64, heads=4)
  (gat2): GATConv(256, 64, heads=4)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=2, bias=True)
  )
)

Total parameters: 143,298


In [21]:
print("\n=== STEP 9: Training Setup ===")

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)

# Loss function (weighted for class imbalance)
class_counts = torch.bincount(data.y)
class_weights = 1.0 / class_counts.float()
class_weights = class_weights / class_weights.sum()
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

print(f"Class weights: Normal={class_weights[0]:.4f}, Suspicious={class_weights[1]:.4f}")


=== STEP 9: Training Setup ===


Class weights: Normal=0.0010, Suspicious=0.9990


In [23]:
# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# torch.cuda.empty_cache()

In [24]:
print("\n=== STEP 10: Training ===")

def train_one_epoch():
    model.train()
    optimizer.zero_grad()
    
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = criterion(out[train_mask], data.y[train_mask])
    
    loss.backward()
    optimizer.step()
    
    return loss.item()

def evaluate(mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index, data.edge_attr)
        pred = out[mask].argmax(dim=1)
        
        y_true = data.y[mask].cpu().numpy()
        y_pred = pred.cpu().numpy()
        
        recall = recall_score(y_true, y_pred, zero_division=0)
        precision = precision_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        
        return recall, precision, f1
    
#Training loop
best_val_recall = 0
patience = 20
patience_counter = 0

for epoch in range(1, 201):
    loss = train_one_epoch()
    
    if epoch % 10 == 0:
        train_recall, train_precision, train_f1 = evaluate(train_mask)
        val_recall, val_precision, val_f1 = evaluate(val_mask)
        
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | "
              f"Val Recall: {val_recall:.4f} | Val Precision: {val_precision:.4f} | Val F1: {val_f1:.4f}")
        
        # Save best model
        if val_recall > best_val_recall:
            best_val_recall = val_recall
            torch.save(model.state_dict(), 'best_model.pth')
            patience_counter = 0
            print(f"  → New best recall: {best_val_recall:.4f}")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch}")
            break


=== STEP 10: Training ===
Epoch 010 | Loss: 158.3768 | Val Recall: 1.0000 | Val Precision: 0.0012 | Val F1: 0.0025
  → New best recall: 1.0000
Epoch 020 | Loss: 72.2170 | Val Recall: 0.8791 | Val Precision: 0.0012 | Val F1: 0.0024
Epoch 030 | Loss: 54.7298 | Val Recall: 0.4444 | Val Precision: 0.0013 | Val F1: 0.0026
Epoch 040 | Loss: 28.4909 | Val Recall: 0.4281 | Val Precision: 0.0013 | Val F1: 0.0027
Epoch 050 | Loss: 22.4792 | Val Recall: 0.5425 | Val Precision: 0.0016 | Val F1: 0.0031
Epoch 060 | Loss: 29.9950 | Val Recall: 0.9641 | Val Precision: 0.0015 | Val F1: 0.0030
Epoch 070 | Loss: 16.3367 | Val Recall: 0.5490 | Val Precision: 0.0018 | Val F1: 0.0036
Epoch 080 | Loss: 7.9677 | Val Recall: 0.4771 | Val Precision: 0.0021 | Val F1: 0.0041
Epoch 090 | Loss: 11.5234 | Val Recall: 0.8170 | Val Precision: 0.0019 | Val F1: 0.0037
Epoch 100 | Loss: 4.4207 | Val Recall: 0.8758 | Val Precision: 0.0018 | Val F1: 0.0035
Epoch 110 | Loss: 2.7123 | Val Recall: 0.5163 | Val Precision: 0.0

In [54]:
print("\n=== STEP 11: Final Evaluation ===")

# Load best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on test set
test_recall, test_precision, test_f1 = evaluate(test_mask)

print(f"\nTest Set Results:")
print(f"  Recall:    {test_recall:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  F1 Score:  {test_f1:.4f}")

# Confusion matrix
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, data.edge_attr)
    pred = out[test_mask].argmax(dim=1)
    
    y_true = data.y[test_mask].cpu().numpy()
    y_pred = pred.cpu().numpy()
    
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\nConfusion Matrix:")
    print(f"  True Negatives:  {tn}")
    print(f"  False Positives: {fp}")
    print(f"  False Negatives: {fn}")
    print(f"  True Positives:  {tp}")

    print(f"\nRates:")
    print(f"  TPR (Recall): {tp/(tp+fn):.4f}")
    print(f"  TNR: {tn/(tn+fp):.4f}")
    print(f"  FPR: {fp/(fp+tn):.4f}")
    print(f"  FNR: {fn/(fn+tp):.4f}")


=== STEP 11: Final Evaluation ===

Test Set Results:
  Recall:    0.9481
  Precision: 0.0054
  F1 Score:  0.0107


/tmp/ipykernel_3551652/2306298191.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))



Confusion Matrix:
  True Negatives:  220876
  False Positives: 27052
  False Negatives: 8
  True Positives:  146

Rates:
  TPR (Recall): 0.9481
  TNR: 0.8909
  FPR: 0.1091
  FNR: 0.0519
